In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
from sklearn.preprocessing import MinMaxScaler
from sklearn import ensemble

In [ ]:
!pip install xgboost

In [77]:
import xgboost as xgb

ModuleNotFoundError: No module named 'xgboost'

In [4]:
energy_df = pd.read_csv('Combined_Data_for_model_final_2.csv')

In [5]:
energy_df.head()

,Unnamed: 0,Date,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility
0,0,2016-01-01 00:15:00,4099.37,11.0,7.0,76.36,0.0,0.0,27.4,1.0
1,1,2016-01-01 00:30:00,3995.64,11.0,7.0,76.36,0.0,0.0,27.4,1.0
2,2,2016-01-01 00:45:00,3954.80,11.0,7.0,76.36,0.0,0.0,27.4,1.0
3,3,2016-01-01 01:00:00,3904.93,11.0,8.0,81.75,0.0,0.0,27.4,1.0
4,4,2016-01-01 01:15:00,3654.68,11.0,8.0,81.75,0.0,0.0,27.4,1.0


In [6]:
year_list = []
month_list = []
for time in energy_df['Date']:
    year = str(time)[:4]
    month = str(time)[5:7]
    year_list.append(year)
    month_list.append(month)

In [7]:
energy_df['month'] = [int(m) for m in month_list]

In [8]:
energy_df

,Unnamed: 0,Date,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month
0,0,2016-01-01 00:15:00,4099.37,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1
1,1,2016-01-01 00:30:00,3995.64,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1
2,2,2016-01-01 00:45:00,3954.80,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1
3,3,2016-01-01 01:00:00,3904.93,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1
4,4,2016-01-01 01:15:00,3654.68,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...
210426,210426,2021-12-31 22:45:00,4712.55,10.0,8.0,87.38,0.0,0.0,27.4,2.0,12
210427,210427,2021-12-31 23:00:00,4649.58,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12
210428,210428,2021-12-31 23:15:00,4571.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12
210429,210429,2021-12-31 23:30:00,4510.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12


In [9]:
months_unique = set(month_list)
months_unique

{'01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'}

In [10]:
energy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210431 entries, 0 to 210430
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         210431 non-null  int64  
 1   Date               210431 non-null  object 
 2   Haryana Load (MW)  210431 non-null  float64
 3   temp               210431 non-null  float64
 4   dew                210431 non-null  float64
 5   humidity           210431 non-null  float64
 6   windspeed          210431 non-null  float64
 7   winddir            210431 non-null  float64
 8   cloudcover         210431 non-null  float64
 9   visibility         210431 non-null  float64
 10  month              210431 non-null  int64  
dtypes: float64(8), int64(2), object(1)
memory usage: 17.7+ MB


In [11]:
def get_season(TimeSeries):
    
    season_list = []
    
    year_list = []
    month_list = []
    for time in TimeSeries:
        year = str(time)[:4]
        month = str(time)[5:7]
        year_list.append(year)
        month_list.append(month) 
       
    for m in month_list:
        if m=='01' or m=='02' or m=='12':
            season_list.append(0)   # Winter
        elif m=='03' or m=='04' or m=='05' or m=='06':
            season_list.append(1) #Spring-Summer
        elif m=='07' or m=='08' or m=='09':
            season_list.append(2) #Monsoon
        elif m=='10' or m=='11':
            season_list.append(3) #Autumn
            
    #season = pd.Series(season_list)
    return season_list

In [12]:
season = get_season(energy_df['Date'])

In [13]:
energy_df['Season'] = season

In [14]:
hour_list = []
for time in energy_df['Date']:
    hour = str(time)[11:13]
    hour_list.append(hour)
hour_list

['00',
 '00',
 '00',
 '01',
 '01',
 '01',
 '01',
 '02',
 '02',
 '02',
 '02',
 '03',
 '03',
 '03',
 '03',
 '04',
 '04',
 '04',
 '04',
 '05',
 '05',
 '05',
 '05',
 '06',
 '06',
 '06',
 '06',
 '07',
 '07',
 '07',
 '07',
 '08',
 '08',
 '08',
 '08',
 '09',
 '09',
 '09',
 '09',
 '10',
 '10',
 '10',
 '10',
 '11',
 '11',
 '11',
 '11',
 '12',
 '12',
 '12',
 '12',
 '13',
 '13',
 '13',
 '13',
 '14',
 '14',
 '14',
 '14',
 '15',
 '15',
 '15',
 '15',
 '16',
 '16',
 '16',
 '16',
 '17',
 '17',
 '17',
 '17',
 '18',
 '18',
 '18',
 '18',
 '19',
 '19',
 '19',
 '19',
 '20',
 '20',
 '20',
 '20',
 '21',
 '21',
 '21',
 '21',
 '22',
 '22',
 '22',
 '22',
 '23',
 '23',
 '23',
 '23',
 '00',
 '00',
 '00',
 '00',
 '01',
 '01',
 '01',
 '01',
 '02',
 '02',
 '02',
 '02',
 '03',
 '03',
 '03',
 '03',
 '04',
 '04',
 '04',
 '04',
 '05',
 '05',
 '05',
 '05',
 '06',
 '06',
 '06',
 '06',
 '07',
 '07',
 '07',
 '07',
 '08',
 '08',
 '08',
 '08',
 '09',
 '09',
 '09',
 '09',
 '10',
 '10',
 '10',
 '10',
 '11',
 '11',
 '11',
 '11',

In [15]:
energy_df['Hour of day'] = [int(hour) for hour in hour_list]

In [16]:
energy_df

,Unnamed: 0,Date,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,Season,Hour of day
0,0,2016-01-01 00:15:00,4099.37,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0
1,1,2016-01-01 00:30:00,3995.64,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0
2,2,2016-01-01 00:45:00,3954.80,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0
3,3,2016-01-01 01:00:00,3904.93,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,1
4,4,2016-01-01 01:15:00,3654.68,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210426,210426,2021-12-31 22:45:00,4712.55,10.0,8.0,87.38,0.0,0.0,27.4,2.0,12,0,22
210427,210427,2021-12-31 23:00:00,4649.58,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23
210428,210428,2021-12-31 23:15:00,4571.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23
210429,210429,2021-12-31 23:30:00,4510.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23


In [17]:
day_list = []
energy_df['Date'] = pd.to_datetime(energy_df['Date'])
for time in energy_df['Date']:
    day = time.weekday()
    day_list.append(day)
energy_df['Day of Week'] = day_list

In [18]:
energy_df

,Unnamed: 0,Date,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,Season,Hour of day,Day of Week
0,0,2016-01-01 00:15:00,4099.37,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4
1,1,2016-01-01 00:30:00,3995.64,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4
2,2,2016-01-01 00:45:00,3954.80,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4
3,3,2016-01-01 01:00:00,3904.93,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,1,4
4,4,2016-01-01 01:15:00,3654.68,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210426,210426,2021-12-31 22:45:00,4712.55,10.0,8.0,87.38,0.0,0.0,27.4,2.0,12,0,22,4
210427,210427,2021-12-31 23:00:00,4649.58,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4
210428,210428,2021-12-31 23:15:00,4571.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4
210429,210429,2021-12-31 23:30:00,4510.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4


In [19]:
energy_df['Is Weekend'] = [day == 5 or day == 6 for day in energy_df['Day of Week']]

In [20]:
energy_df

,Unnamed: 0,Date,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,Season,Hour of day,Day of Week,Is Weekend
0,0,2016-01-01 00:15:00,4099.37,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,False
1,1,2016-01-01 00:30:00,3995.64,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,False
2,2,2016-01-01 00:45:00,3954.80,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,False
3,3,2016-01-01 01:00:00,3904.93,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,1,4,False
4,4,2016-01-01 01:15:00,3654.68,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,1,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210426,210426,2021-12-31 22:45:00,4712.55,10.0,8.0,87.38,0.0,0.0,27.4,2.0,12,0,22,4,False
210427,210427,2021-12-31 23:00:00,4649.58,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,False
210428,210428,2021-12-31 23:15:00,4571.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,False
210429,210429,2021-12-31 23:30:00,4510.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,False


In [21]:
clean_weekend = []
for b in energy_df['Is Weekend']:
    if b==True:
        clean_weekend.append(1)
    elif b==False:
        clean_weekend.append(0)
energy_df['Is Weekend Clean'] = clean_weekend

In [22]:
energy_df

,Unnamed: 0,Date,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,Season,Hour of day,Day of Week,Is Weekend,Is Weekend Clean
0,0,2016-01-01 00:15:00,4099.37,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,False,0
1,1,2016-01-01 00:30:00,3995.64,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,False,0
2,2,2016-01-01 00:45:00,3954.80,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,False,0
3,3,2016-01-01 01:00:00,3904.93,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,1,4,False,0
4,4,2016-01-01 01:15:00,3654.68,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,1,4,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210426,210426,2021-12-31 22:45:00,4712.55,10.0,8.0,87.38,0.0,0.0,27.4,2.0,12,0,22,4,False,0
210427,210427,2021-12-31 23:00:00,4649.58,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,False,0
210428,210428,2021-12-31 23:15:00,4571.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,False,0
210429,210429,2021-12-31 23:30:00,4510.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,False,0


In [23]:
energy_df = energy_df.drop(columns=['Is Weekend'])

In [24]:
energy_df.corr()

,Unnamed: 0,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,Season,Hour of day,Day of Week,Is Weekend Clean
Unnamed: 0,1.000000,0.150388,-0.040199,0.061319,0.111563,-0.043855,-0.057483,0.068373,0.021891,0.166419,0.096422,0.000442,-0.001181,-0.002004
Haryana Load (MW),0.150388,1.000000,0.504668,0.617157,0.016372,0.087153,-0.029084,0.287901,0.325990,0.177620,0.259011,0.114884,-0.041774,-0.056105
temp,-0.040199,0.504668,1.000000,0.538345,-0.588910,0.362062,0.134242,0.239265,0.692226,0.054718,0.363203,0.219212,-0.005567,-0.007670
dew,0.061319,0.617157,0.538345,1.000000,0.327262,0.073790,-0.165958,0.554322,0.303009,0.237475,0.462137,-0.013864,0.005373,-0.000827
humidity,0.111563,0.016372,-0.588910,0.327262,1.000000,-0.327792,-0.303280,0.284030,-0.499458,0.140681,0.006129,-0.280852,0.010816,0.007714
windspeed,-0.043855,0.087153,0.362062,0.073790,-0.327792,1.000000,0.483971,0.179585,0.345374,-0.164791,-0.082914,0.141720,0.014333,0.012012
winddir,-0.057483,-0.029084,0.134242,-0.165958,-0.303280,0.483971,1.000000,-0.084202,0.134194,-0.113523,-0.100639,0.134540,-0.006979,0.002149
cloudcover,0.068373,0.287901,0.239265,0.554322,0.284030,0.179585,-0.084202,1.000000,0.117772,-0.000521,0.096489,0.018671,-0.005450,-0.012627
visibility,0.021891,0.325990,0.692226,0.303009,-0.499458,0.345374,0.134194,0.117772,1.000000,-0.016544,0.225875,0.191557,-0.005567,0.003059
month,0.166419,0.177620,0.054718,0.237475,0.140681,-0.164791,-0.113523,-0.000521,-0.016544,1.000000,0.579282,-0.000013,0.001226,0.000959


In [25]:
energy_df

,Unnamed: 0,Date,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,Season,Hour of day,Day of Week,Is Weekend Clean
0,0,2016-01-01 00:15:00,4099.37,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,0
1,1,2016-01-01 00:30:00,3995.64,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,0
2,2,2016-01-01 00:45:00,3954.80,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,0
3,3,2016-01-01 01:00:00,3904.93,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,1,4,0
4,4,2016-01-01 01:15:00,3654.68,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210426,210426,2021-12-31 22:45:00,4712.55,10.0,8.0,87.38,0.0,0.0,27.4,2.0,12,0,22,4,0
210427,210427,2021-12-31 23:00:00,4649.58,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,0
210428,210428,2021-12-31 23:15:00,4571.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,0
210429,210429,2021-12-31 23:30:00,4510.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,0


In [26]:
# energy_df.to_csv('Complete_Dataset_added_Calendar_features_2.csv')

In [27]:
energy_df

,Unnamed: 0,Date,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,Season,Hour of day,Day of Week,Is Weekend Clean
0,0,2016-01-01 00:15:00,4099.37,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,0
1,1,2016-01-01 00:30:00,3995.64,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,0
2,2,2016-01-01 00:45:00,3954.80,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,0
3,3,2016-01-01 01:00:00,3904.93,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,1,4,0
4,4,2016-01-01 01:15:00,3654.68,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210426,210426,2021-12-31 22:45:00,4712.55,10.0,8.0,87.38,0.0,0.0,27.4,2.0,12,0,22,4,0
210427,210427,2021-12-31 23:00:00,4649.58,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,0
210428,210428,2021-12-31 23:15:00,4571.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,0
210429,210429,2021-12-31 23:30:00,4510.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,0


In [28]:
energy_df.drop(columns = {'Date', 'Unnamed: 0'}, inplace=True)

In [29]:
energy_df

,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,Season,Hour of day,Day of Week,Is Weekend Clean
0,4099.37,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,0
1,3995.64,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,0
2,3954.80,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,0
3,3904.93,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,1,4,0
4,3654.68,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210426,4712.55,10.0,8.0,87.38,0.0,0.0,27.4,2.0,12,0,22,4,0
210427,4649.58,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,0
210428,4571.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,0
210429,4510.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,0


In [30]:
df_train = energy_df.iloc[:int(energy_df.shape[0]*0.7)]

In [31]:
df_train

,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,Season,Hour of day,Day of Week,Is Weekend Clean
0,4099.37,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,0
1,3995.64,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,0
2,3954.80,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,0
3,3904.93,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,1,4,0
4,3654.68,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
147296,4396.38,18.2,16.5,89.40,6.4,85.0,55.7,2.0,3,1,8,5,1
147297,4479.17,18.2,16.5,89.40,6.4,85.0,55.7,2.0,3,1,8,5,1
147298,4493.16,18.2,16.5,89.40,6.4,85.0,55.7,2.0,3,1,8,5,1
147299,4543.41,19.0,17.0,88.19,11.2,100.0,37.1,2.0,3,1,9,5,1


In [32]:
df_test = energy_df[df_train.shape[0]:]

In [33]:
df_test

,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,Season,Hour of day,Day of Week,Is Weekend Clean
147301,4637.64,19.0,17.0,88.19,11.2,100.0,37.1,2.0,3,1,9,5,1
147302,4652.91,19.0,17.0,88.19,11.2,100.0,37.1,2.0,3,1,9,5,1
147303,4636.97,21.0,16.0,73.11,13.0,90.0,37.1,3.0,3,1,10,5,1
147304,4569.88,21.0,16.0,73.11,13.0,90.0,37.1,3.0,3,1,10,5,1
147305,4514.03,21.0,16.0,73.11,13.0,90.0,37.1,3.0,3,1,10,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210426,4712.55,10.0,8.0,87.38,0.0,0.0,27.4,2.0,12,0,22,4,0
210427,4649.58,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,0
210428,4571.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,0
210429,4510.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,0


In [34]:
scaler = MinMaxScaler()

In [35]:
X = df_train.drop(columns='Haryana Load (MW)')

In [36]:
X

,temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,Season,Hour of day,Day of Week,Is Weekend Clean
0,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,0
1,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,0
2,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,0,0,4,0
3,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,1,4,0
4,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
147296,18.2,16.5,89.40,6.4,85.0,55.7,2.0,3,1,8,5,1
147297,18.2,16.5,89.40,6.4,85.0,55.7,2.0,3,1,8,5,1
147298,18.2,16.5,89.40,6.4,85.0,55.7,2.0,3,1,8,5,1
147299,19.0,17.0,88.19,11.2,100.0,37.1,2.0,3,1,9,5,1


In [74]:
y_train = energy_df['Haryana Load (MW)'][:df_train.shape[0]]

In [75]:
y_train

0         4099.37
1         3995.64
2         3954.80
3         3904.93
4         3654.68
           ...   
147296    4396.38
147297    4479.17
147298    4493.16
147299    4543.41
147300    4595.77
Name: Haryana Load (MW), Length: 147301, dtype: float64

In [39]:
mmo = scaler.fit_transform(X, y_train)

In [71]:
type(mmo)

numpy.ndarray

In [72]:
mmo.shape

(147301, 12)

In [42]:
reg = ensemble.GradientBoostingRegressor()

In [43]:
reg.fit(mmo, y_train)

GradientBoostingRegressor()

In [46]:
df_test.drop(columns='Haryana Load (MW)', inplace=True)

C:\Users\HP\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [47]:
df_test

,temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,Season,Hour of day,Day of Week,Is Weekend Clean
147301,19.0,17.0,88.19,11.2,100.0,37.1,2.0,3,1,9,5,1
147302,19.0,17.0,88.19,11.2,100.0,37.1,2.0,3,1,9,5,1
147303,21.0,16.0,73.11,13.0,90.0,37.1,3.0,3,1,10,5,1
147304,21.0,16.0,73.11,13.0,90.0,37.1,3.0,3,1,10,5,1
147305,21.0,16.0,73.11,13.0,90.0,37.1,3.0,3,1,10,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...
210426,10.0,8.0,87.38,0.0,0.0,27.4,2.0,12,0,22,4,0
210427,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,0
210428,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,0
210429,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,23,4,0


In [48]:
mpo = scaler.fit_transform(df_test)

In [49]:
predictions = reg.predict(mpo)

In [58]:
predictions = pd.DataFrame(predictions)

In [52]:
y_test = energy_df['Haryana Load (MW)'].iloc[df_train.shape[0]:]

In [53]:
y_test

147301    4637.64
147302    4652.91
147303    4636.97
147304    4569.88
147305    4514.03
           ...   
210426    4712.55
210427    4649.58
210428    4571.39
210429    4510.39
210430    4449.94
Name: Haryana Load (MW), Length: 63130, dtype: float64

In [55]:
import sklearn

In [60]:
r2s = sklearn.metrics.r2_score(y_test, predictions[0])
r2s

0.5960383445481411

In [68]:
params = {
    "n_estimators": 1000,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.1,
    "loss": "squared_error",
}

In [69]:
reg2 = ensemble.GradientBoostingRegressor(params)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass loss={'n_estimators': 1000, 'max_depth': 4, 'min_samples_split': 5, 'learning_rate': 0.1, 'loss': 'squared_error'} as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [76]:
reg2.fit(mmo, y_train)

TypeError: unsupported format string passed to dict.__format__

In [67]:
predictions = reg2.predict(mpo)

AttributeError: 'GradientBoostingRegressor' object has no attribute 'estimators_'